# 使用私有数据微调ChatGLM3 #

In [1]:
import os
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'
# train_data_path = 'data/zhouyi_dataset_20250618_214633.csv'
# train_data_path = 'data/zhouyi_dataset_20240118_152413.csv'
# train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'
train_data_path = 'data/zhouyi_dataset_handmade.csv'
eval_data_path = None
seed = 8
max_input_length = 512
max_output_length = 1536
lora_rank = 16
lora_alpha = 32
lora_dropout = 0.05
prompt_text = ''

## 数据处理

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset) #dataset是字典


/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 8
    })
})


In [4]:
dataset['train'].features # features 是数据集特征描述，包含每个列的名称和类型

{'content': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [5]:
dataset['train']['content'] # content是键

['乾卦', '坤卦', '水雷屯卦', '山水蒙卦', '水天需卦', '天水讼卦', '地水师卦', '水地比卦']

In [6]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dateset."
    
    # 生成一组随机数 picks 用作 dataset索引
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    # 生成 df表格
    df = pd.DataFrame(dataset[picks]) # 返回picks列表的行索引写到表格df
    for column,typ in dataset.features.items():# 固定格式目的是转换数据集中的分类标签和分类标签序列
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
        
    

In [7]:
show_random_elements(dataset["train"],num_examples=5)

,content,summary
0,水雷屯卦,屯卦原文：屯。元，亨，利，贞。勿用，有攸往，利建侯。象曰：云，雷，屯；君子以经纶。白话文解释：屯卦大吉大利，吉利的占卜。不利于出门。有利于建国封侯。《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽和雷的威严治理国事。《断易天机》解：屯卦坎上震下，为坎宫二世卦。屯卦显示困难，动而逢险，需刚毅果敢方为吉。北宋易学家邵雍解：万物始生，开始困难；先劳后逸，苦尽甘来。得此卦者，身处困境，宜守不宜进，需辛劳克难，初难后解。台湾国学大儒傅佩荣解：时运宜守，财运创业艰难，家宅初婚不和，身体需保元气。传统解卦：异卦（下震上坎），震为雷动，坎为雨险。雷雨交加，环境险恶。“屯”指万物始生，艰难险阻中顺时应运，终将欣荣。
1,山水蒙卦,蒙卦原文：蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。象曰：山下出泉，蒙。君子以果行育德。白话文解释：蒙卦通泰。不是我求幼稚之人，而是幼稚之人求我。初次占卜被告知，轻慢占卜则不再告知。占卜吉利。《象辞》说：卦象为山下有泉，取法于山泉果敢坚毅，培养品德。《断易天机》解：蒙卦艮上坎下，象征蒙昧，主疑惑不前，多忧愁，凶兆。北宋易学家邵雍解：智慧未开，犹豫不决，需顺师友教导启智。台湾国学大儒傅佩荣解：时运蓄德出世，财运矿业果决吉，家宅君子居吉，婚姻之始，身体驱邪保安。传统解卦：异卦（下坎上艮），山下有险仍前进，为蒙昧，把握时机行动恰时，启蒙通达之象。大象：蒙为昏无所见，宜启蒙。
2,乾卦,乾卦原文\n乾。元，亨，利，贞。\n象曰：天行健，君子以自强不息。\n\n白话文解释\n乾卦：大吉大利，吉利的贞卜。\n《象辞》说：天道刚健，运行不已。君子观此卦象，从而以天为法，自强不息。\n\n《断易天机》解\n乾象征天，六阳爻构成乾卦，为《易经》六十四卦之首。纯阳刚建，其性刚强，其行劲健，大通而至正，兆示大通而有利，但须行正道，方可永远亨通。\n\n北宋易学家邵雍解\n刚健旺盛，发育之功；完事顺利，谨防太强。\n得此卦者，天行刚健，自强不息，名利双收之象，宜把握机会，争取成果。女人得此卦则有过于刚直之嫌。\n\n传统解卦\n这个卦是同卦（下乾上乾）相叠。象征天，喻龙（德才的君子），又象征纯粹的阳和健，表明兴盛强健。乾卦是根据万物变通的道理，以“元、亨、利、贞”为卦辞，表示吉祥如意，教导人遵守天道的德行。
3,天水讼卦,讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
4,水天需卦,需卦原文：需。有孚，光亨，贞吉。利涉大川。象曰：云上于天，需；君子以饮食宴乐。白话文解释：需卦代表俘虏，大吉大利，适宜涉水过河。《象辞》说：上卦为坎，象征云；下卦为乾，象征天。云聚天上，待降雨，君子观此卦，宜宴饮安乐，待时而动。《断易天机》解：需卦坎上乾下，象征踌躇期待，刚强面对险阻，宜等待，涉大川利。北宋易学家邵雍解：遇阻不进，大器晚成，需耐心等待。台湾国学大儒傅佩荣解：时运需耐心等待，财运资本未集，家宅平安，身体调饮食以健康。传统解卦：异卦（下乾上坎），刚逢险，宜稳健，观时待变，必成功。


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [9]:
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """
    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input',None) and example['input'].strip():
        question += f'\n{example["input"]}'
        
    # 构建答案文本
    answer = example['summary']
    
    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)
    
    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:
        q_ids = q_ids[:max_input_length - 2] # 保留空间给gmask和bos标记
    if len(a_ids) > max_output_length - 1:
        a_ids = a_ids[:max_output_length - 1] # 保留空间给eos标记
        
    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2
    
    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]
    
    return {'input_ids':input_ids,'labels':labels}

In [10]:
column_names = dataset['train'].column_names
# 为了保持dataset的格式，使用 Dataset.map 方法。直接给 map 函数传入一个函数，这个函数传入 dataset 的一个样本，然后返回映射后的结果。
tokenized_dataset = dataset['train'].map(
    lambda example:tokenize_func(example, tokenizer),
    batched=False,
    remove_columns=column_names
)

In [11]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
# 调用 flatten_indices() 可以消除执行 dataset.map() 时产生的副作用，确保你的数据集像普通的 list 一样按顺序访问。
tokenized_dataset = tokenized_dataset.flatten_indices()

In [12]:
import torch 
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """
    
    def __init__(self, pad_token_id:int, max_length:int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length
        
    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str,torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)
        
        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d #计算需要填充的长度
            # 田间填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = lbel[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))
            
        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)
        
        return{'input_ids': input_ids, 'labels': labels}

In [13]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [14]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_tpye='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False # silence the warnings. Please re-enable for inference!

Loading checkpoint shards: 100%|██████████| 7/7 [00:06<00:00,  1.02it/s]
/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [16]:
target_modules

['query_key_value']

In [17]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [18]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


## QLoRA微调模型

In [19]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 50


output_dir = f'models/{model_name_or_path}-epoch{train_epochs}-{timestamp}'

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                # 输出目录
    per_device_train_batch_size=1,        # 每个设备的训练批量大小
    gradient_accumulation_steps=1,        # 梯度累计步数
    learning_rate=1e-3,                   # 学习率
    num_train_epochs=train_epochs,        # 训练轮数
    lr_scheduler_type='linear',           # 学习率调度器类型
    warmup_ratio=0.1,                     # 预热比例
    logging_steps=1,                      # 日志记录步数
    save_strategy='steps',                # 模型记录策略
    save_steps=10,                        # 模型保存步数
    optim='adamw_torch',                  # 优化器类型
    fp16=True                             # 是否使用混合精度训练
)

In [21]:
trainer = Trainer(
    model=qlora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [22]:
trainer.train()

Step,Training Loss
1,4.867100
2,3.596800
3,4.649500
4,4.028100
5,4.698500
6,4.468000
7,3.635000
8,4.150000
9,3.786000
10,4.316000


TrainOutput(global_step=400, training_loss=0.2951196938470093, metrics={'train_runtime': 373.6129, 'train_samples_per_second': 1.071, 'train_steps_per_second': 1.071, 'total_flos': 4003180407398400.0, 'train_loss': 0.2951196938470093, 'epoch': 50.0})

In [23]:
trainer.model.save_pretrained(output_dir)